In [39]:
# Assignment 11

# Recommendation System 

In [40]:
import pandas as pd

df = pd.read_csv('anime.csv')
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [42]:
df.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [43]:
# imputing missing values

df['genre'] =df['genre'].fillna('Unknown')
df['type'] = df['type'].fillna('Unknown')

df['rating'] = df['rating'].fillna(df['rating'].median())

In [44]:
# object to numerical 
# handling missing

df['episodes'] = pd.to_numeric(df['episodes'],errors='coerce')

df['episodes'] = df['episodes'].fillna(df['episodes'].median()).astype(int)

In [45]:
df.isnull().sum() , df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12294 non-null  object 
 3   type      12294 non-null  object 
 4   episodes  12294 non-null  int64  
 5   rating    12294 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 672.5+ KB


(anime_id    0
 name        0
 genre       0
 type        0
 episodes    0
 rating      0
 members     0
 dtype: int64,
 None)

In [46]:
# Normalizing
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_rating = scaler.fit_transform(df[['rating']])
scaled_members = scaler.fit_transform(df[['members']])

In [47]:
# Feature Extraction
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['genre'])

In [48]:
# combining features 

import numpy as np

tfidf_dense = tfidf_matrix.toarray()

combined_features = np.hstack((tfidf_dense, scaled_rating, scaled_members))

In [49]:
# Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(combined_features, combined_features)

In [50]:
# Recommendation func
def get_recommendations(title, similarity=similarity):
    
    idx = df[df['name'] == title].index[0]

    sim_scores = list(enumerate(similarity[idx]))                             # Get the pairwise similarity scores of all anime with that anime

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)         # Sort the anime based on the similarity scores

    sim_scores = sim_scores[1:11]                                             # Get the scores of the 10 most similar anime

    anime_indices = [i[0] for i in sim_scores]                                # Get the anime indices

    return df[['anime_id','name']].iloc[anime_indices]                        # Return the top 10 most similar anime

    

In [51]:
recommendations = get_recommendations('Harmonie')

recommendations.reset_index(drop=True, inplace=True)

print(f"Top 10 Recommendations :\n{recommendations}")

Top 10 Recommendations :
   anime_id                                               name
0     10067                     Angel Beats!: Another Epilogue
1     32281                                     Kimi no Na wa.
2      8728  Code Geass: Hangyaku no Lelouch - Kiseki no Bi...
3       547                        Wind: A Breath of Heart OVA
4       546                       Wind: A Breath of Heart (TV)
5     18881  Code Geass: Hangyaku no Lelouch - Kiseki no Bi...
6     14669              Aura: Maryuuin Kouga Saigo no Tatakai
7       344                                   Futatsu no Spica
8     10389                                   Momo e no Tegami
9      3001                                         Moyashimon


#### Interview Questions

* Cosine similarity :
   - This method calculates the cosine of the angle between the feature vectors of two anime.
   - cosine similarity closer to 1 (smaller angle) means the anime are more similar.
     
1. - User CF :
       - Finds users who are similar to the target user & recommends items that those similar users liked.
   - Item CF :
       - Finds items similar to what the user liked & recommends those items.

2. Collaborative Filtering (CF) :

   - technique used by recommendation systems to make automatic FILTERING the interests of a user by collecting preferences or taste information from many users (COLLABRATING).
   - The underlying assumption is that if two users have agreed in the past, they are likely to agree again in the future.